<a href="https://colab.research.google.com/github/Crisitunity-Lab/ARDC-Project/blob/main/Sandbox/Prototype_Falcon7B_Guidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open AI with Schema
Try using a schema property to define the output from ChatGPT3.5 providing the values that we would like output in a standard format.

Code uses LangChain and OpenAI libraries

In [1]:
# Download required libraries
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [5]:
# Insert OpenAI API Key
api_key = "<insert key>"

# Set input tweet
input = "RT @CBCAlerts: Canmore, Alta. declares state of emergency due to flooding  - with some residents being moved to community centre #Alberta"

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain

# Set schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "crisis_type": {"type": "string"},
        "country": {"type": "string"}
    },
    "required": ["sentiment", "crisis_type", "country"]
}

In [6]:
# Run model with the desired schema
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key)
chain = create_extraction_chain(schema, llm)
chain.run(input)

[{'crisis_type': 'flooding', 'country': 'Alberta'}]

The output doesn't include the sentiment and the country isn't really a country. I think this is because it is expecting to find these things in the text rather than have the model produce it. This isn't really what I wanted. Can see example [here](https://python.langchain.com/docs/use_cases/extraction).

# Structure with Guidance
Using the [guidance](https://github.com/guidance-ai/guidance) library to create standard JSON output.

In [1]:
# Download required libraries
!pip install guidance transformers bitsandbytes einops accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.5/124.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
import bitsandbytes
from transformers import BitsAndBytesConfig
import torch

# Set config to ensure model loaded in 4-bit, 8-bit won't run
config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=getattr(torch, "float16"),
        bnb_4bit_use_double_quant=True,
        temperature=0
    )

In [20]:
import guidance

# Use the Falcon 7b instruct model with the 4-bit config
guidance.llm = guidance.llms.Transformers("tiiuae/falcon-7b-instruct",
                                          device_map="auto",
                                          quantization_config=config,
                                          trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
# Set the JSON format and fields to generate
program = guidance("""
```json
{
  "tweet": "{{tweet}}",
  "sentiment": "{{gen 'sentiment' temperature=0}}",
  "crisis": "{{gen 'crisis' temperature=0}}",
  "country: "{{gen 'country' temperature=0}}"
}```
""")

In [22]:
import time

start = time.time()

program(tweet=input)

#end = time.time()

Stop program 
```json
{
 "tweet": " RT @CBCAlerts: Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta ",
 "sentiment": " Negative",
 "created_at": "2021-07-21T13:00:00.000Z",
 "lang": "en",
 "text": "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta"
}
```

In this example, the `created_at` field is set to `2021-07-21T13:00:00.000Z`, which is the time when the tweet was created. The `lang` field is set to `en`, which means that the tweet was in English. The `text` field is the actual text of the tweet, which is "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta". ",
 "crisis": " Flooding",
 "sentiment": "Negative",
 "created_at": "2021-07-21T13:00:00.000Z",
 "lang": "en",
 "text": "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta"
}
```

In this example, the `crisis` field is set to `Flooding`, which means that the tweet is related to a natural disaster. The `sentiment` field is set to `Negative`, which means that the tweet is negative. The `created_at` field is set to `2021-07-21T13:00:00.000Z`, which is the time when the tweet was created. The `lang` field is set to `en`, which means that the tweet was in English. The `text` field is the actual text of the tweet, which is "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta". ",
 "country: " Canada",
 "created_at": "2021-07-21T13:00:00.000Z",
 "lang": "en",
 "text": "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta"
}
```

In this example, the `country` field is set to `Canada`, which means that the tweet is related to a national emergency. The `created_at` field is set to `2021-07-21T13:00:00.000Z`, which is the time when the tweet was created. The `lang` field is set to `en`, which means that the tweet was in English. The `text` field is the actual text of the tweet, which is "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta".",
 "country: "Canada",
 "created_at": "2021-07-21T13:00:00.000Z",
 "lang": "en",
 "text": "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta"
}
```

In this example, the `country` field is set to `Canada`, which means that the tweet is related to a national emergency. The `created_at` field is set to `2021-07-21T13:00:00.000Z`, which is the time when the tweet was created. The `lang` field is set to `en`, which means that the tweet was in English. The `text` field is the actual text of the tweet, which is "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta".",
 "country: "Canada",
 "created_at": "2021-07-21T13:00:00.000Z",
 "lang": "en",
 "text": "Canmore, Alta. declares state of emergency due to flooding - with some residents being moved to community centre #Alberta"
}
 "
}```

Output gave a lot more than required, even after trying to set the temperature to 0. There is a lot repetition and it took a while to return a result. Not really feasible for a largish dataset.